In [ ]:
import pip

!pip install Pillow

In [ ]:
"""
pip install opencv-contrib-python==3.4.2.16
"""
import base64
import json
import pprint
import cv2
import numpy as np
from finalProject.classes.yolo import Yolo
from finalProject.utils.drawing.draw import draw_final_results
from finalProject.utils.images.imagesUtils import Image
from finalProject.utils.keyPoints.AlgoritamKeyPoints import create_keypoints_descriptors, sift_keypoints_detection
from finalProject.utils.matchers.Matchers import compare_between_descriptors, flann_matcher
from finalProject.utils.preprocessing.preprocess import read_frames_from_video, check_frames_exist, reduce_noise, removeRemovalColor
from finalProject.utils.tracking.TrackingByYolo import source_detection_by_yolo, tracking_by_yolo
import matplotlib.pyplot as plt
import random
import pandas as pd
import io
from PIL import Image
import os
import datetime


def sobel(img):
    sobelx = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=5)
    sobely = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=5)
    sobel_intensity = cv2.sqrt(cv2.addWeighted(cv2.pow(sobelx, 2.0), 1.0, cv2.pow(sobely, 2.0), 1.0, 0.0))
    return sobel_intensity


def sobel_keypoints(image):
    sobelImage = sobel(image)
    # norm
    image8bit = cv2.normalize(sobelImage, None, 0, 255, cv2.NORM_MINMAX).astype('uint8')
    kp, des = sift_keypoints_detection(image8bit)
    return kp, des, image8bit


def forward(frames):
    outputs = []
    for frame in frames:
        frame.gray_image = sobel(frame.gray_image)
        key_des_image = sobel_keypoints(frame.gray_image)
        outputs.append(key_des_image)
    return outputs


def saveImage(image, folder_name, image_name):
    path = "./reports/"+str(folder_name)+"/"+image_name+".png"
    cv2.imwrite(path, image)


def convert(folder_name, counter):
    def convertImagetoIds(output, counter):
        saveImage(output[0], folder_name,"comparison_"+counter)
        saveImage(output[1], folder_name,"source_"+counter)
        saveImage(output[2], folder_name,"target_"+counter)
        return output[3:]
    return convertImagetoIds


def exportToExcel(outputs):
    curr_time = datetime.datetime.now()
    print(curr_time.strftime("%d-%m-%Y_%Hh-%Mm"))

    folder_name = "folderReport_" + str(curr_time.strftime("%d-%m-%Y_%-I%p-%Mm"))
    path = "./reports/" + folder_name
    os.makedirs(path, mode=0o777, exist_ok=False)

    counts = []
    for i, v in enumerate(outputs):
        counts.append(str(i))

    excel_table = list(map(convert(folder_name, str(i)), outputs, counts))
    print("excel_table: ")
    print(excel_table)
    df2 = pd.DataFrame(np.array(excel_table),
                       columns=['source_keypoints', 'target_keypoints', 'total_matches', 'accuracy'])
    # df2.to_excel("./reports/" + folder_name + "/report_" + folder_name + ".xls")
    df2.to_excel("./reports/" + folder_name + "/report_" + folder_name + ".xls", float_format="%.2f", index=False,)


def cross_correction(key_des_image_source, key_des_image_target, threshold=0.8):
    (k1, d1, source_img) = key_des_image_source
    (k2, d2, target_img) = key_des_image_target
    match = flann_matcher(d1, d2, threshold=threshold)

    if len(match) == 0:
        return None

    output = cv2.drawMatchesKnn(source_img, k1, target_img, k2, match, outImg=None)
    acc = len(match)/len(k1)
    return [output, source_img, target_img, len(k1), len(k2), len(match), min(acc, 1)]  # item = (outputs,source_img,target_img,len(k1),len(k2),acc..)


def sort_by_acc_func(output_row):
    return output_row[6]


def save_people_frames(source_person, target_people):
    path = "./reports/"+str(folder_name)+"/"+image_name+".png"
    cv2.imwrite(path, image)


def plotSquare(images, _titles, threshold, each_column=10):
    number_images = len(images)
    rows = (number_images // each_column)
    index = 1

    print(rows)
    fig, axes = plt.subplots(rows, each_column, figsize=(48, 24),dpi=300)
    fig.subplots_adjust(hspace=0.3, wspace=0.05)

    for ax,image,title in zip(axes.flat, images, _titles):
        ax.imshow(image)
        ax.set_title(title)
        index += 1

    rand = random.uniform(0, 1)
    pathX = "./figures/fig_test"+str(rand)+"_thresh_"+str(threshold)+".svg"
    print("saving on", pathX)
    plt.savefig(pathX, dpi=fig.dpi)
    plt.clf()
    plt.close()

In [3]:
# init yolo
yolo = Yolo()
yolo.initYolo()
pp = pprint.PrettyPrinter(indent=4)


[INFO] loading YOLO from disk...


In [4]:
def run(threshold=0.8):

    with open('./config.txt') as file_json:
        config = json.load(file_json)

        # compute_runtime = config["output"]["runtime"]

        # source
        source_frames = read_frames_from_video(config["source"])
        if not check_frames_exist(source_frames):
            print("problem with the input of source video.")
            exit(0)

        source_person = source_detection_by_yolo(source_frames, yolo,
                                                 is_video=config["source"]["isVideo"],
                                                 config=config["source"])
        if source_person is None:
            print("failed to detect a person on source video")
            exit(0)

        # target
        target_frames = read_frames_from_video(config["target"])
        if not check_frames_exist(target_frames):
            print("problem with the input of target video.")
            exit(0)

        target_people = tracking_by_yolo(target_frames, yolo, is_video=config["target"]["isVideo"],config=config["target"])

        cv2.destroyAllWindows()

        # source
        sources_output = forward(source_person.frames)  # array of (key,des,image) of all source's frames

        targets_outputs = []  # array of targets , each target has an array of frames
        for target_person in target_people:
            target_output = forward(target_person.frames)  # array of (key,des,image) of all target_person's frames
            targets_outputs.append(target_output)

        outputs = []  # each frame

        for source_frame in sources_output:
            for target in targets_outputs:
                for target_frame in target:  # threshold for match ! bigger - more lines
                    cross = cross_correction(source_frame, target_frame, threshold)
                    if cross is not None:
                        outputs.append(cross)

        outputs.sort(reverse=True, key=sort_by_acc_func)
        exportToExcel(outputs[:10])

        print("source_person.frames[0].frame_parts: ")
        print(source_person.frames[0].frame_parts)
        # save_people_frames(source_person, target_people)

#         titles = ["test"] * (len(plots) - 1)
#         plotSquare(plots, titles,threshold)
#         print("finish")

In [5]:
for thresh in [0.8]:
    run(threshold=thresh)


NameError: name 'folder_name' is not defined